In [ ]:
"""
This file produces an organized, simplified table from raw behavioral .csv files generated by the ABET behavioral testing software. 
Using Pandas to generate a data frame from the raw data, we then use a series of functions to pull out relevant 
behavioral events within each trial.

This code was written by Patrick Piantadosi & Kendall Coden, from the Laboratory of Behavioral and Genomic Neuroscience @ NIAAA. 
"""

In [3]:
# Import relevant packages
import pandas as pd
import numpy as np

In [4]:
# read in behavioral data as .csv
# eventually will write this more flexibly as a function (so users can give a filename externally)
data = pd.read_csv("74 12042019.csv") 

In [5]:
#Time values < 0 are housekeeping variables reported by the software. Safe to filter these out so they do not get counted as trials, etc.
filter_time_zero = data.Evnt_Time != 0
filter_time_zero.values
data = data[filter_time_zero]

In [6]:
#logic for indicating new trials (which will later be used to groupby), adds a new column to the data frame (data) that counts the number of trials based on trial starts
is_new_trial = (data.Item_Name == "Forced-Choice Trials Begin") | (data.Item_Name == "Free-Choice Trials begin")
data["is_new_trial"] = is_new_trial
data["is_new_trial"].value_counts()
data["trial_num"] = np.cumsum(data["is_new_trial"])
data["trial_num"].max()

164

In [7]:
"""
The function below (make_table) makes a table based upon input from the functions defined below
"""
def make_table(data):
    # add new functions here
    BlockNum = get_block(data)
    trial_type = get_trial_type(data)
    rew_type = rew_size(data)
    shocker = shock(data)
    choice_time = choiceTime(data)
    collect_time = collectTime(data)
    start_time = startTime(data)


    return pd.Series({"BlockNum": BlockNum,
                     "trial_type": trial_type,
                     "rew_type": rew_type,
                     "shock": shocker,
                     "choice_time": choice_time,
                     "collect_time": collect_time,
                     "start_time": start_time})

In [8]:
"""
This function checks the data for which of the 3 trial blocks ("Sessions") the mouse is currently in, and then returns an 
integer value depending on the block of a specific trial

"""
def get_block(data):
    is_block_1 = data[data["Item_Name"] == "Session1"]["Arg1_Value"]
    is_block_2 = data[data["Item_Name"] == "Session2"]["Arg1_Value"]
    is_block_3 = data[data["Item_Name"] == "Session3"]["Arg1_Value"]
    
    if is_block_1.any():
        return 1
    elif is_block_2.any():
        return 2
    elif is_block_3.any():
        return 3

In [9]:
"""
This function checks what size reward the mouse selected for a given trial, 
and assigns the value (speed of the peristaltic pump, in ms) to the trial
"""

def rew_size(data):
    if len(data.loc[data["Item_Name"] == "Feeder #2"]["Arg1_Value"]) == 0:
        return np.nan
    elif len(data.loc[data["Item_Name"] == "Feeder #2"]["Arg1_Value"]) == 1:
        return (data.loc[data["Item_Name"] == "Feeder #2"]["Arg1_Value"]).values[0]
    else:
        raise ValueError(f"more than one reward {data.trial_num.iloc[0].values[0]}")


In [10]:
"""
Each trial block is made up of Forced-Choice or Free-Choice trials, this function checks which 
trial type the mouse is in, and assigns that value to the trial 
"""

def get_trial_type(data):
    if (data["Item_Name"] == "Forced-Choice Trials Begin").sum():
        return "Forced"
    elif (data["Item_Name"] == "Free-Choice Trials begin").sum():
        return "Free"

In [11]:
"""
Some trials result in a minor footshock, this function checks whether the 
footshock occured or not, and assigns that value to the trial
"""

def shock(data):
    if len(data.loc[data["Item_Name"] == "shock_on_off"]["Arg1_Value"]) == 0:
        return np.nan
    elif len(data.loc[data["Item_Name"] == "shock_on_off"]["Arg1_Value"]) == 1:
        return(data.loc[data["Item_Name"] == "shock_on_off"]["Arg1_Value"]).values[0]

In [12]:
"""
Events are timestamped in the behavioral software. This function returns the time at which a mouse initiates a trial.
"""

def startTime(data):
    if len(data.loc[data["Item_Name"].str.contains("Trials Begin" or "Trials begin")]) == 0:
        return np.nan
    elif len(data.loc[data["Item_Name"].str.contains("Trials Begin" or "Trials begin")]) == 1:
        return(data.loc[data["Item_Name"].str.contains("Trials Begin" or "Trials begin")]["Evnt_Time"]).values[0]

In [13]:
"""
Events are timestamped in the behavioral software. This function returns the time at which a mouse chooses the
large or small reward.
"""

def choiceTime(data):
    if len(data.loc[data["Item_Name"] == "Feeder #2"]["Arg1_Value"]) == 0:
        return np.nan
    elif len(data.loc[data["Item_Name"] == "Feeder #2"]["Arg1_Value"]) == 1:
        return(data.loc[data["Item_Name"] == "Feeder #2"]["Evnt_Time"]).values[0]

In [14]:
"""
Events are timestamped in the behavioral software. This function returns the time at which a mouse collects the reward.
"""
def collectTime(data):
    if len(data.loc[data["Item_Name"].str.contains("Reward Retrieved")]) == 0:
        return np.nan
    elif len(data.loc[data["Item_Name"].str.contains("Reward Retrieved")]) == 1:
        return(data.loc[data["Item_Name"].str.contains("Reward Retrieved")]["Evnt_Time"]).values[0]

In [15]:
grouped_data = data.groupby("trial_num")
result = grouped_data.apply(make_table)

In [16]:
result

,BlockNum,trial_type,rew_type,shock,choice_time,collect_time,start_time
trial_num,,,,,,,
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Forced,0.3,NaN,77.730,80.718,60.893
2,1.0,Forced,1.2,0.0,92.805,94.180,89.491
3,1.0,Forced,1.2,0.0,106.316,107.639,102.515
4,NaN,Forced,NaN,NaN,NaN,NaN,115.703
5,1.0,Forced,0.3,NaN,160.323,161.949,153.728
6,1.0,Forced,1.2,0.0,172.678,173.818,170.712
7,1.0,Forced,1.2,0.0,189.132,190.290,187.368
8,1.0,Forced,0.3,NaN,212.068,213.712,201.322


In [26]:
"""
Will eventually need functions to filter based on data type, can use .loc method to accomplish this
The code below filters the newly generated table "result" by whether the mouse chose a large or small reward
"""
large_rew_only = result.loc[result["rew_type"] == 1.2]

In [27]:
large_rew_only

,BlockNum,trial_type,rew_type,shock,choice_time,collect_time,start_time
trial_num,,,,,,,
2,1.0,Forced,1.2,0.0,92.805,94.180,89.491
3,1.0,Forced,1.2,0.0,106.316,107.639,102.515
6,1.0,Forced,1.2,0.0,172.678,173.818,170.712
7,1.0,Forced,1.2,0.0,189.132,190.290,187.368
11,1.0,Free,1.2,0.0,256.532,257.649,NaN
12,1.0,Free,1.2,0.0,272.724,273.955,NaN
13,1.0,Free,1.2,0.0,286.633,287.949,NaN
14,1.0,Free,1.2,0.0,299.751,300.933,NaN
15,1.0,Free,1.2,0.0,313.158,314.468,NaN
